In [3]:
import sys
import sys
sys.path.append('../../')
from omnicell.constants import *

from omnicell.models.gears.gears import GEARS
import omnicell.models.gears.pertdata as pertdata_new
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.pertdata import PertData
from omnicell.data.catalogue import Catalogue
from omnicell.models.gears.gears import GEARS
import scanpy as sc
import torch
import numpy as np


In [4]:


DATASET_NAME = 'repogle_k562_essential_raw'
dd = Catalogue.get_dataset_details(DATASET_NAME)
adata = sc.read(dd.path)






In [5]:

adata.obs["condition"] = adata.obs[dd.pert_key]
adata.obs["cell_type"] = adata.obs[dd.cell_key]
perts = [p for p in adata.obs["condition"].unique() if p != dd.control]
adata.obs["condition"] = adata.obs["condition"].replace({dd.control:"ctrl"})
adata.obs["condition"] = adata.obs["condition"].replace({p:p+"+ctrl" for p in perts})
adata.var["gene_name"] = adata.var_names




In [6]:



print(f"Data loaded from {dd.path}")
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
print(f"Data normalized and log transformed")

device = 'cuda' if torch.cuda.is_available() else 'cpu'



Data loaded from /orcd/data/omarabu/001/Omnicell_datasets/repogle_k562_essential_raw/K562_essential_raw_singlecell_01.h5ad
Data normalized and log transformed


In [ ]:


pert_data = pertdata_old.PertData('./data') # specific saved folder
print(f"Saving data in ./data")

pert_data.new_data_process(dataset_name = DATASET_NAME, adata = adata, skip_calc_de=False) # specific dataset name and adata object
print(f"Data processed and saved in {pert_data.data_path}")
pert_data.prepare_split(split = 'no_test', seed = 1) # get data split with seed
print(f"Data split with seed 1")
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader
print(f"Data loader prepared")


In [ ]:



gears_model = GEARS(pert_data, device = device, 
                        weight_bias_track = False, 
                        proj_name = 'pertnet', 
                        exp_name = 'pertnet')

gears_model.model_initialize(hidden_size = 64)

gears_model.tunable_parameters()



In [ ]:

print(f"Training model")
gears_model.train(epochs = 1, lr = 1e-3)


gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')


res = gears_model.predict([['RPL15']])

np.savez('res.npz', res)



In [7]:
pert_data_new = pertdata_new.PertData('./data_new') # specific saved folder


print(f"Saving data in ./data")

pert_data_new.new_data_process(dataset_name = DATASET_NAME, adata = adata, skip_calc_de=True) # specific dataset name and adata object
print(f"Data processed and saved in {pert_data_new.data_path}")
pert_data_new.prepare_split(split = 'no_test', seed = 1) # get data split with seed
print(f"Data split with seed 1")
pert_data_new.get_dataloader(batch_size = 32, test_batch_size = 128) # prepare data loader
print(f"Data loader prepared")


Downloading...
100%|██████████| 9.46M/9.46M [00:00<00:00, 17.6MiB/s]


Saving data in ./data
Starting new data processing for repogle_k562_essential_raw
Getting DE Genes
Got DE genes for repogle_k562_essential_raw: 3.50 seconds


Downloading...
100%|██████████| 559k/559k [00:00<00:00, 2.00MiB/s]


Test
Creating pyg object for each cell in the data...
Creating dataset file...


  0%|          | 1/2058 [00:02<1:33:01,  2.71s/it]

...

Creating 119 cell graphs for MED9+ctrl: 0.00 seconds
create_cell_graph_dataset call duration: MED9+ctrl: 2.70 seconds


  0%|          | 9/2058 [00:26<1:50:48,  3.24s/it]

Creating 1996 cell graphs for RPL3+ctrl: 0.33 seconds
create_cell_graph_dataset call duration: RPL3+ctrl: 4.25 seconds


  0%|          | 9/2058 [00:28<1:48:02,  3.16s/it]


KeyboardInterrupt: 

In [10]:

gears_model = GEARS(pert_data_new, device = device, 
                        weight_bias_track = False, 
                        proj_name = 'pertnet_new', 
                        exp_name = 'pertnet_new')

gears_model.model_initialize(hidden_size = 64)

gears_model.tunable_parameters()




Downloading...
100%|██████████| 60.7M/60.7M [00:28<00:00, 2.10MiB/s]
Extracting tar file...
Done!


{'hidden_size': 'hidden dimension, default 64',
 'num_go_gnn_layers': 'number of GNN layers for GO graph, default 1',
 'num_gene_gnn_layers': 'number of GNN layers for co-expression gene graph, default 1',
 'decoder_hidden_size': 'hidden dimension for gene-specific decoder, default 16',
 'num_similar_genes_go_graph': 'number of maximum similar K genes in the GO graph, default 20',
 'num_similar_genes_co_express_graph': 'number of maximum similar K genes in the co expression graph, default 20',
 'coexpress_threshold': 'pearson correlation threshold when constructing coexpression graph, default 0.4',
 'uncertainty': 'whether or not to turn on uncertainty mode, default False',
 'uncertainty_reg': 'regularization term to balance uncertainty loss and prediction loss, default 1',
 'direction_lambda': 'regularization term to balance direction loss and prediction loss, default 1'}

In [11]:
next(iter(gears_model.dataloader["train_loader"]))

DataBatch(x=[160000, 1], y=[32, 5000], pert_idx=[32], de_idx=[32], pert=[32], batch=[160000], ptr=[33])

In [12]:

print(f"Training model")
gears_model.train(epochs = 1, lr = 1e-3)




gears_model.save_model('test_model')
gears_model.load_pretrained('test_model')


res = gears_model.predict([['RPL15']])

np.savez('res.npz', res)


Start Training...
Epoch 1 Step 1 Train Loss: 0.2994


Training model
Dataloader element: DataBatch(x=[160000, 1], y=[32, 5000], pert_idx=[32], de_idx=[32], pert=[32], batch=[160000], ptr=[33])


Epoch 1 Step 51 Train Loss: 0.3393
Epoch 1 Step 101 Train Loss: 0.3529
Epoch 1 Step 151 Train Loss: 0.3464
Epoch 1: Train Overall MSE: 0.0013 Validation Overall MSE: 0.0005. 
Train Top 20 DE MSE: 0.0299 Validation Top 20 DE MSE: 0.0026. 
Done!
Done! No test dataloader detected.


In [13]:
res

{'RPL15': array([ 0.25589207, -0.0418413 ,  0.08747627, ..., -0.00059464,
         0.00094366, -0.00029467], dtype=float32)}